In [2]:
# Libraries
import os
import pandas as pd
pd.options.display.max_rows = 100 
pd.options.display.max_columns = None
from dateutil import tz
from datetime import datetime

# 1. Preview

In [4]:
path = "API_EXTRACTED_DATA_PATH"

## Habitat

### Capacity (number of spaces)

In [3]:
directory = path+"habitat/"
entity = "habitat_spaces_"

dfs = []

for filename in os.listdir(directory):
    if filename.startswith(entity):
        # Read each habitat_spaces file
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, sep=';')
        
        # Add the date id as a column to the df
        df['date'] = int(filename[len(entity):len(entity)+8])
        
        # Locate habitat_spacetypes file corresponding to habitat_spaces file date
        types_filename = f"habitat_spacetypes_{filename[len(entity):]}"
        typespath = os.path.join(directory, types_filename)
        # Add usetype column to the df
        spacetypes_df = pd.read_csv(typespath, sep=';')
        merged_df = pd.merge(df, spacetypes_df[['id', 'usetype']], left_on='type', right_on='id', how='left')
        df['spacetype'] = merged_df['usetype']

        # Locate habitat_floors file corresponding to habitat_spaces file date
        floors_filename = f"habitat_floors_{filename[len(entity):]}"
        floorspath = os.path.join(directory, floors_filename)
        # Add building column to the df
        floors_df = pd.read_csv(floorspath, sep=';')
        merged_df = pd.merge(df, floors_df[['id', 'name', 'building']], left_on='floor', right_on='id', how='left')
        df['building'] = merged_df['building']

        # Locate habitat_buildings file corresponding to habitat_spaces file date
        buildings_filename = f"habitat_buildings_{filename[len(entity):]}"
        buildingspath = os.path.join(directory, buildings_filename)
        # Add address column to the df
        buildings_df = pd.read_csv(buildingspath, sep=';')
        merged_df = pd.merge(df, buildings_df[['id', 'address', 'latitude','longitude']], left_on='building', right_on='id', how='left')
        df['address'] = merged_df['address']
        # Add office location
        df['latitude'] = merged_df['latitude']
        df['longitude'] = merged_df['longitude']

        dfs.append(df)

habitat_spaces = pd.concat(dfs, ignore_index=True)

habitat_spaces

,id,code,type,name,description,entrydate,x,y,floor,capacity,availabilitytype,availabilitydate,date,spacetype,building,address,latitude,longitude
0,3816,C9-P3PP-01,11,PuestoPruebas,Puesto Trabajo,2023-01-30T09:58:01,0.000,0.00,11,1,NoSensor,2023-01-30T09:58:01,20240201,Desk,3,Gran Vía de Hortaleza,40.70,-3.50
1,4470,M-F29N-037,11,M-F29N-037,Desk,2023-04-24T11:32:39,641.997,4061.39,18,1,NoSensor,2023-04-24T11:32:39,20240201,Desk,5,"55 E. Monroe, 29th Floor",41.88,-87.62
2,4471,M-F29N-038,11,M-F29N-038,Desk,2023-04-24T11:32:40,609.000,4061.22,18,1,NoSensor,2023-04-24T11:32:40,20240201,Desk,5,"55 E. Monroe, 29th Floor",41.88,-87.62
3,4472,M-F29N-039,11,M-F29N-039,Desk,2023-04-24T11:32:40,641.997,4121.39,18,1,NoSensor,2023-04-24T11:32:40,20240201,Desk,5,"55 E. Monroe, 29th Floor",41.88,-87.62
4,4473,M-F29N-040,11,M-F29N-040,Desk,2023-04-24T11:32:41,609.000,4121.22,18,1,NoSensor,2023-04-24T11:32:41,20240201,Desk,5,"55 E. Monroe, 29th Floor",41.88,-87.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41546,7833,T9-P3-c91,54,9-P3-c91,Taquilla,2024-02-05T16:07:33,83.930,10.58,11,1,NoSensor,2024-02-05T16:07:33,20240208,Desk,3,Gran Vía de Hortaleza,40.70,-3.50
41547,7834,T9-P3-c92,54,9-P3-c92,Taquilla,2024-02-05T16:07:33,80.610,10.58,11,1,NoSensor,2024-02-05T16:07:33,20240208,Desk,3,Gran Vía de Hortaleza,40.70,-3.50
41548,7835,T9-P3-c93,54,9-P3-c93,Taquilla,2024-02-05T16:07:34,80.610,10.58,11,1,NoSensor,2024-02-05T16:07:34,20240208,Desk,3,Gran Vía de Hortaleza,40.70,-3.50
41549,7836,T9-P3-c94,54,9-P3-c94,Taquilla,2024-02-05T16:07:34,80.610,10.58,11,1,NoSensor,2024-02-05T16:07:34,20240208,Desk,3,Gran Vía de Hortaleza,40.70,-3.50


#### Filter by space type

In [4]:
print(habitat_spaces['spacetype'].unique())

['Desk' 'Room' 'Parking']


In [5]:
# Delete spaces of type Parking
habitat_spaces = habitat_spaces[habitat_spaces['spacetype'] != 'Parking']

print(habitat_spaces['description'].unique())
# There are still some spaces which are not workplaces, they should be deleted
habitat_spaces = habitat_spaces[~habitat_spaces['description'].str.contains('Sala Descanso|Sala Lactancia|Sala Comedor|Taquilla')]

['Puesto Trabajo' 'Desk' 'TBD Desk' 'Desk - I&C Autocad'
 'Desk- I&C Autocad' 'Desk ' 'Desk - O&M SCADA' 'Posto de Trabalho'
 'Puesto Director' 'Puesto Director ' 'Posto Diretor'
 'Puesto Colaboración' 'Phone Booth' 'Phone Room' 'Cabine Individual'
 'Sala de reuniones' 'Sala de Reuniones' 'Sala de reunioes' 'Despacho'
 'Sala diretor' 'Puesto Confidente' 'Cecoer Desk' 'Conference Room'
 'Office' 'Concentration-Silence' 'Open Room' 'Sala Agora' 'Sala/Despacho'
 'Sala Descanso' 'Sala Lactancia' 'Sala Comedor' 'Puesto Director L'
 'Puesto Operativo' 'Puesto Colaboración sin pantalla'
 'Puesto Operativo Elevable' 'Puesto Operativo Elevable apoyo cervical'
 'Puesto Operativo apoyo cervical' 'Taquilla']


#### Filter by address

In [6]:
# Only interested in Mesena campus: Gran Vía de Hortaleza
habitat_spaces['address'].unique()

array(['Gran Vía de Hortaleza', '55 E. Monroe, 29th Floor',
       'Rua das Olimpiadas, 134'], dtype=object)

In [7]:
# Filter by address
habitat_spaces = habitat_spaces[habitat_spaces['address'].str.contains('Hortaleza')]
print(f'Number of rows: {len(habitat_spaces)}')

Number of rows: 19760


In [8]:
# Print office location coordinates for meteorological data
unique_coordinates = habitat_spaces[['latitude', 'longitude']].drop_duplicates()
print(unique_coordinates)

   latitude  longitude
0      40.7       -3.5


##### Now that the dataset is filtered, the relevant information is contained in the capacity variable, as the interest lies in the capacity of CAMPUS for each day: how many workstations are available for employees in a day.

In [9]:
# Group by date and add up the capacity of each space
workplaces = habitat_spaces.groupby('date')['capacity'].sum().reset_index(name='capacity')
workplaces

,date,capacity
0,20240201,3462
1,20240202,3462
2,20240203,3462
3,20240204,3462
4,20240205,3462
5,20240206,3462
6,20240207,3462
7,20240208,3462


### Reservations (number of employees that reserve a workstation)

In [10]:
directory = path+"habitat/"
entity = "habitat_reservations_"

dfs = []

for filename in os.listdir(directory):
    if filename.startswith(entity):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, index_col=0)            
        dfs.append(df)


habitat_reservations = pd.concat(dfs, ignore_index=True)
habitat_reservations.drop_duplicates()

habitat_reservations

,id,owner,space,status,startDate,endDate,name,description,hidden
0,901746,90034001,1270,Finished,2024-01-30T23:00:00,2024-01-31T23:00:22,NaN,Continuous Reservation,False
1,921294,50073309,4669,Finished,2024-01-30T23:00:00,2024-01-31T23:00:22,NaN,Continuous Reservation,False
2,924385,50082035,3641,Finished,2024-01-30T23:00:00,2024-01-31T23:00:22,NaN,Continuous Reservation,False
3,952242,90191670,4297,Finished,2024-01-30T23:00:00,2024-01-31T23:00:22,NaN,Continuous Reservation,False
4,952330,50076875,4338,Finished,2024-01-30T23:00:00,2024-01-31T23:00:22,NaN,Continuous Reservation,False
...,...,...,...,...,...,...,...,...,...
34813,1353514,50063225,4428,Cancelled,2024-02-08T21:45:00,2024-02-08T23:29:59,NaN,NaN,False
34814,1353515,50063225,4433,Cancelled,2024-02-08T21:45:00,2024-02-08T23:29:59,NaN,NaN,False
34815,1353518,50063225,4428,Cancelled,2024-02-08T21:45:00,2024-02-08T22:29:59,NaN,NaN,False
34816,1353491,90208242,4431,Finished,2024-02-08T22:00:00,2024-02-08T22:30:13,NaN,NaN,False


#### Filter by space
Keep only reservations regarding desired spaces (the ones resulting from the transformations done just above)

In [11]:
# Keep only reservations of workplaces in Mesena office
habitat_reservations = habitat_reservations[habitat_reservations['space'].isin(habitat_spaces['id'])]
# habitat_reservations

##### Transform date format
Convert the given time in UTC to local in order to have correct date time ranges

In [12]:
def convert_to_local(date_str):
    
    from_zone = tz.tzutc()
    to_zone = tz.gettz('Europe/Madrid')

    # Remove T and decimals from datetimes
    date_str = date_str.replace('T',' ').split('.')[0]
    utc = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    
    # Tell the datetime object that it is in UTC time zone
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone and remove time as it is not useful
    central = utc.astimezone(to_zone).strftime('%Y-%m-%d %H:%M:%S')
    
    return central

In [13]:
# Time is in UTC, convert to local using convert_to_local function
habitat_reservations.loc[:, 'startDate'] = habitat_reservations['startDate'].apply(convert_to_local)
habitat_reservations.loc[:, 'endDate'] = habitat_reservations['endDate'].apply(convert_to_local)

# Eliminate reservations from days which are out of the scope
habitat_reservations = habitat_reservations[habitat_reservations['startDate'] >= '2024-02-01 00:00:00']

# Create date column
habitat_reservations.loc[:,'date'] = habitat_reservations['startDate'].str.replace('-','')
habitat_reservations.loc[:,'date'] = habitat_reservations['date'].str[0:8]
habitat_reservations['date'] = habitat_reservations['date'].astype('int')

In [14]:
# Dato raro, no en BBDD
habitat_reservations = habitat_reservations[habitat_reservations['owner'] != 700]

In [15]:
habitat_reservations

,id,owner,space,status,startDate,endDate,name,description,hidden,date
2672,897322,50023723,4620,Cancelled,2024-02-01 00:00:00,2024-02-01 23:59:59,NaN,Continuous Reservation,False,20240201
2673,898131,90197035,4889,Cancelled,2024-02-01 00:00:00,2024-02-01 23:59:59,NaN,Continuous Reservation,False,20240201
2674,898220,90018468,4912,Cancelled,2024-02-01 00:00:00,2024-02-01 23:59:59,NaN,Continuous Reservation,False,20240201
2675,898311,90164109,1561,Cancelled,2024-02-01 00:00:00,2024-02-01 23:59:59,NaN,Continuous Reservation,False,20240201
2676,898400,90164109,1560,Cancelled,2024-02-01 00:00:00,2024-02-01 23:59:59,NaN,Continuous Reservation,False,20240201
...,...,...,...,...,...,...,...,...,...,...
34788,1353184,90022782,3734,Cancelled,2024-02-08 17:15:00,2024-02-08 17:44:59,NaN,NaN,False,20240208
34790,1352438,90193803,3300,Finished,2024-02-08 18:00:00,2024-02-08 19:00:16,NaN,NaN,False,20240208
34792,1353225,90014344,1611,Finished,2024-02-08 18:00:00,2024-02-08 18:30:14,NaN,NaN,False,20240208
34793,1353257,90033833,1729,Finished,2024-02-08 18:00:00,2024-02-08 19:00:16,NaN,NaN,False,20240208


In [16]:
# Group by date and add up the number of employees
num_reservations = habitat_reservations.groupby('date')['owner'].nunique().reset_index(name='workplace_reservations')
num_reservations

,date,workplace_reservations
0,20240201,2174
1,20240202,2148
2,20240205,2141
3,20240206,2166
4,20240207,2184
5,20240208,2171


## Lenel

### Total occupancy (Events: accesses to offices) 

In [ ]:
directory = path+"lenel/"
entity = "lenel_loggedevents_"

dfs = []

for filename in os.listdir(directory):
    if filename.startswith(entity):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, sep=';')
        
        # Add the date as a column to the df
        date = filename[len(entity):len(entity)+8]
        df['date'] = int(date)

        # Locate lenel_cardholders file corresponding to the lenel_loggedevents file date
        cardholders_filename = f"lenel_cardholders_{date}"
        for f in os.listdir(directory):
            if f.startswith(cardholders_filename):
                cardholders_path = os.path.join(directory, f)
                # Read lenel_cardholders file and merge employee_id (ssno) column with lenel_loggedevents
                cardholders_df = pd.read_csv(cardholders_path, sep=';')
                merged_df = pd.merge(df, cardholders_df[['id', 'ssno']], left_on='cardholder_key', right_on='id', how='left')
                
        dfs.append(merged_df)


lenel_accesses = pd.concat(dfs, ignore_index=True)
# Rename employee_id column
lenel_accesses = lenel_accesses.rename(columns={'ssno': 'id_employee'})

lenel_accesses

#### Filter by access and Mesena offices

In [18]:
# Remove not granted accesses
lenel_accesses = lenel_accesses[lenel_accesses['access_result'] == 2]
# Remove not completed accesses
lenel_accesses = lenel_accesses[lenel_accesses['cardholder_entered'] == True]
# Keep only accesses from Mesena campus
lenel_accesses = lenel_accesses[lenel_accesses['controller_name'].str.contains('_MES_')]
# lenel_accesses

In [19]:
# Keep unique list of employees' id
lenel_accesses = lenel_accesses[['date','cardholder_key','id_employee']].drop_duplicates()
lenel_accesses

,date,cardholder_key,id_employee
0,20240201,3013,50202244Y
3,20240201,3066,30006376
11,20240201,12188,50071896
13,20240201,21256,53438310H
22,20240201,2541,53824460K
...,...,...,...
178358,20240208,20270,CAMPUS-P0087
178371,20240208,1817,50017049
179466,20240208,3193,90107394
180261,20240208,2367,30001537


In [20]:
# Not all users that enter campus are registered as cardholders
lenel_accesses.isnull().sum()

date                0
cardholder_key      0
id_employee       390
dtype: int64

In [21]:
# Total number of granted and completed accesses per day to Mesena campus
occupancy_total = lenel_accesses.groupby('date')['cardholder_key'].count().reset_index(name='occupancy_total')

occupancy_total

,date,occupancy_total
0,20240201,2589
1,20240202,2317
2,20240203,61
3,20240204,44
4,20240205,2657
5,20240206,2695
6,20240207,2658
7,20240208,2613


#### Filtering by permission

In [22]:
permissions_df = pd.read_csv(path+"lenel/permissions_20240201_20240208.csv")

# Keep only users permissions in Mesena
permissions_df = permissions_df[permissions_df['cod_sede']=='MESENA']

# Convert date types
permissions_df['fec_fecha'] = permissions_df['fec_fecha'].str.replace('-','').astype(int)
permissions_df['id_empleado'] = permissions_df['id_empleado'].astype(str)
permissions_df

,fec_fecha,id_empleado,de_permisos,id_sede,cod_sede,de_permiso_extra
0,20240201,10102638,",desks,pooler_moto,room,incidents,emergency,se...",1,MESENA,NaN
1,20240201,30000092,",incidents,emergency,search,notifications,news...",1,MESENA,NaN
2,20240201,30000100,",incidents,emergency,search,notifications,news...",1,MESENA,NaN
3,20240201,30000140,",incidents,emergency,search,notifications,news...",1,MESENA,NaN
4,20240201,30000142,",incidents,emergency,search,notifications,news...",1,MESENA,NaN
...,...,...,...,...,...,...
38613,20240208,90209406,",desks,parking,room,incidents,emergency,search...",1,MESENA,NaN
38614,20240208,90209407,",desks,parking,room,incidents,emergency,search...",1,MESENA,NaN
38615,20240208,90209517,",desks,pooler_moto,room,incidents,emergency,se...",1,MESENA,NaN
38616,20240208,90209541,",desks,pooler_moto,room,incidents,emergency,se...",1,MESENA,NaN


In [23]:
perm_merged_df = pd.merge(lenel_accesses, permissions_df[['fec_fecha', 'id_empleado', 'de_permisos', 'de_permiso_extra']], 
                          left_on=['date', 'id_employee'], 
                          right_on=['fec_fecha', 'id_empleado'], 
                          how='left')
perm_merged_df

,date,cardholder_key,id_employee,fec_fecha,id_empleado,de_permisos,de_permiso_extra
0,20240201,3013,50202244Y,NaN,NaN,NaN,NaN
1,20240201,3066,30006376,20240201.0,30006376,",incidents,emergency,search,notifications,news...",NaN
2,20240201,12188,50071896,NaN,NaN,NaN,NaN
3,20240201,21256,53438310H,NaN,NaN,NaN,NaN
4,20240201,2541,53824460K,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
15629,20240208,20270,CAMPUS-P0087,NaN,NaN,NaN,NaN
15630,20240208,1817,50017049,20240208.0,50017049,",desks,pooler_moto,room,incidents,emergency,se...",NaN
15631,20240208,3193,90107394,20240208.0,90107394,",desks,pooler_moto,room,incidents,emergency,se...",NaN
15632,20240208,2367,30001537,20240208.0,30001537,",incidents,emergency,search,notifications,news...",NaN


In [24]:
# nulls in id_employee -> accesses of users which are not registered as cardholders
# nulls in id_empleado -> accesses of users which are not registered as cardholders and have no permissions registered
perm_merged_df.isnull().sum()

date                    0
cardholder_key          0
id_employee           390
fec_fecha            2731
id_empleado          2731
de_permisos          2731
de_permiso_extra    13850
dtype: int64

### Occupancy of workplaces

In [25]:
# Function to verify if "desk" or "room" is contained as a permission
def has_workplace_permission(row):
    permissions = str(row['de_permisos'])+str(row['de_permiso_extra'])
    
    if 'desk' in permissions  or 'room' in permissions:
        return True
    else:
        return False

# Create workplace flag
perm_merged_df['flg_workplace'] = perm_merged_df.apply(has_workplace_permission, axis=1)
perm_merged_df

,date,cardholder_key,id_employee,fec_fecha,id_empleado,de_permisos,de_permiso_extra,flg_workplace
0,20240201,3013,50202244Y,NaN,NaN,NaN,NaN,False
1,20240201,3066,30006376,20240201.0,30006376,",incidents,emergency,search,notifications,news...",NaN,False
2,20240201,12188,50071896,NaN,NaN,NaN,NaN,False
3,20240201,21256,53438310H,NaN,NaN,NaN,NaN,False
4,20240201,2541,53824460K,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...
15629,20240208,20270,CAMPUS-P0087,NaN,NaN,NaN,NaN,False
15630,20240208,1817,50017049,20240208.0,50017049,",desks,pooler_moto,room,incidents,emergency,se...",NaN,True
15631,20240208,3193,90107394,20240208.0,90107394,",desks,pooler_moto,room,incidents,emergency,se...",NaN,True
15632,20240208,2367,30001537,20240208.0,30001537,",incidents,emergency,search,notifications,news...",NaN,False


In [26]:
# Número de accesos a puesto por día (desk permission)
lenel_accesses_workplace = perm_merged_df[perm_merged_df['flg_workplace'] == True]
occupancy_workplace = lenel_accesses_workplace.groupby('date')['cardholder_key'].count().reset_index(name='occupancy_workplace')

occupancy_workplace

,date,occupancy_workplace
0,20240201,1986
1,20240202,1861
2,20240203,25
3,20240204,12
4,20240205,2071
5,20240206,2090
6,20240207,2015
7,20240208,1977


### Expected occupancy of restaurant

In [27]:
# Function to verify if "restaurant" is contained as a permission
def has_restaurant_permission(row):
    permissions = str(row['de_permisos'])+str(row['de_permiso_extra'])
    
    if 'restaurant' in permissions:
        return True
    else:
        return False

# Create restaurant flag
perm_merged_df['flg_restaurant'] = perm_merged_df.apply(has_restaurant_permission, axis=1)
perm_merged_df

,date,cardholder_key,id_employee,fec_fecha,id_empleado,de_permisos,de_permiso_extra,flg_workplace,flg_restaurant
0,20240201,3013,50202244Y,NaN,NaN,NaN,NaN,False,False
1,20240201,3066,30006376,20240201.0,30006376,",incidents,emergency,search,notifications,news...",NaN,False,True
2,20240201,12188,50071896,NaN,NaN,NaN,NaN,False,False
3,20240201,21256,53438310H,NaN,NaN,NaN,NaN,False,False
4,20240201,2541,53824460K,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...
15629,20240208,20270,CAMPUS-P0087,NaN,NaN,NaN,NaN,False,False
15630,20240208,1817,50017049,20240208.0,50017049,",desks,pooler_moto,room,incidents,emergency,se...",NaN,True,True
15631,20240208,3193,90107394,20240208.0,90107394,",desks,pooler_moto,room,incidents,emergency,se...",NaN,True,True
15632,20240208,2367,30001537,20240208.0,30001537,",incidents,emergency,search,notifications,news...",NaN,False,True


In [28]:
# Número de accesos a puesto por día (desk permission)
lenel_restaurant = perm_merged_df[perm_merged_df['flg_restaurant'] == True]
exp_occupancy_restaurant = lenel_restaurant.groupby('date')['cardholder_key'].count().reset_index(name='exp_occup_restaurant')

exp_occupancy_restaurant

,date,exp_occup_restaurant
0,20240201,2146
1,20240202,1971
2,20240203,37
3,20240204,22
4,20240205,2202
5,20240206,2245
6,20240207,2157
7,20240208,2123


## Prodelfi

The restaurant service is only offered from Monday to Thursday

In [7]:
df = pd.read_csv(path+'prodelfi/prodelfi_movimientostarjeta_20240207060017.csv', sep=';')
df

,apellidos,cantidad,centrocoste,codigo,codigoexterno,codigo_offline,colectivo,contrata,departamento,descripcioncolectivo,descripcionmaquina,descripcionproducto,empresa,fecha,id1,maquina,maquinamultisite,nombre,numeropedido,persona,precio,producto,productomultisite,responsable,saldoanterior,tarjeta,tipo_ctb,ubicacion
0,NaN,1,NaN,678837,NaN,0,3,NaN,NaN,Colectivo Specialist,Central,Recarga de Saldo Fichero,101,2024-02-06 13:13:07,50042155,999,999,NaN,0,2,-0.5,32002,32002,NaN,0.0,2862,105,NaN
1,NaN,1,NaN,678838,NaN,0,3,NaN,NaN,Colectivo Specialist,TABLET Nº 5 ACCIONA,MENÚ ACCIONA,101,2024-02-06 13:13:07,50042155,12155,999,NaN,95067695,2,0.5,1,32017,NaN,0.5,2862,109,NaN
2,NaN,1,NaN,681476,NaN,0,1,NaN,NaN,Colectivo Director,Central,Recarga de Saldo Fichero,101,2024-02-06 14:39:55,90028400,999,999,NaN,0,3,-0.5,32002,32002,NaN,0.0,3095,105,NaN
3,NaN,1,NaN,681477,NaN,0,1,NaN,NaN,Colectivo Director,TABLET Nº 5 ACCIONA,MENÚ ACCIONA,101,2024-02-06 14:39:55,90028400,12155,999,NaN,95068559,3,0.5,1,32017,NaN,0.5,3095,109,NaN
4,NaN,1,NaN,678789,NaN,0,3,NaN,NaN,Colectivo Specialist,Central,Recarga de Saldo Fichero,101,2024-02-06 13:12:17,90117229,999,999,NaN,0,4,-0.5,32002,32002,NaN,0.0,1364,105,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514,NaN,1,NaN,681589,NaN,0,2,NaN,NaN,Colectivo Manager,TABLET Nº 2 ACCIONA,MENÚ ACCIONA,101,2024-02-06 14:43:51,50095008,12152,999,NaN,95068585,5293,0.5,1,32017,NaN,0.5,8313,109,NaN
4515,NaN,4,NaN,681590,NaN,0,2,NaN,NaN,Colectivo Manager,TABLET Nº 2 ACCIONA,MENÚ ACCIONA,101,2024-02-06 14:43:51,50095008,12152,999,NaN,95068585,5293,0.0,1,32017,NaN,0.0,8313,109,NaN
4516,NaN,1,NaN,679809,NaN,0,6,NaN,NaN,Colectivo Internship,Central,Recarga de Saldo Fichero,101,2024-02-06 13:44:49,50095224,999,999,NaN,0,5297,-0.5,32002,32002,NaN,0.0,9145,105,NaN
4517,NaN,1,NaN,679810,NaN,0,6,NaN,NaN,Colectivo Internship,TABLET Nº 7 ACCIONA,MENÚ ACCIONA,101,2024-02-06 13:44:49,50095224,12157,999,NaN,95068004,5297,0.5,1,32017,NaN,0.5,9145,109,NaN


In [29]:
directory = path+"prodelfi/"
entity = "prodelfi_movimientostarjeta_"

dfs = []

for filename in os.listdir(directory):
    if filename.startswith(entity):
        # Read each movimientostarjeta file
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, sep=';')
        
        # Check df is not empty
        if len(df) > 0:
            # Add the date id as a column to the df
            # Substract one day to get the real date (the file is generated the following day)
            df['date'] = int(filename[len(entity):len(entity)+8]) - 1
            
            dfs.append(df)


prodelfi = pd.concat(dfs, ignore_index=True)

prodelfi

,apellidos,cantidad,centrocoste,codigo,codigoexterno,codigo_offline,colectivo,contrata,departamento,descripcioncolectivo,descripcionmaquina,descripcionproducto,empresa,fecha,id1,maquina,maquinamultisite,nombre,numeropedido,persona,precio,producto,productomultisite,responsable,saldoanterior,tarjeta,tipo_ctb,ubicacion,date
0,NaN,1,NaN,670422,NaN,0,2,NaN,NaN,Colectivo Manager,Central,Recarga de Saldo Fichero,101,2024-02-01 13:14:28,90133632,999,999,NaN,0,14,-0.5,32002,32002,NaN,0.0,7575,105,NaN,20240201
1,NaN,1,NaN,670423,NaN,0,2,NaN,NaN,Colectivo Manager,TABLET Nº 9 MAGDALENA,MENÚ ACCIONA,101,2024-02-01 13:14:28,90133632,12171,999,NaN,95013585,14,0.5,1,32017,NaN,0.5,7575,109,NaN,20240201
2,NaN,1,NaN,672416,NaN,0,2,NaN,NaN,Colectivo Manager,Central,Recarga de Saldo Fichero,101,2024-02-01 14:28:46,90015284,999,999,NaN,0,21,-0.5,32002,32002,NaN,0.0,7988,105,NaN,20240201
3,NaN,1,NaN,672417,NaN,0,2,NaN,NaN,Colectivo Manager,TABLET Nº 4 ACCIONA,MENÚ ACCIONA,101,2024-02-01 14:28:46,90015284,12154,999,NaN,95014326,21,0.5,1,32017,NaN,0.5,7988,109,NaN,20240201
4,NaN,3,NaN,672418,NaN,0,2,NaN,NaN,Colectivo Manager,TABLET Nº 4 ACCIONA,MENÚ ACCIONA,101,2024-02-01 14:28:46,90015284,12154,999,NaN,95014326,21,0.0,1,32017,NaN,0.0,7988,109,NaN,20240201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21498,NaN,2,NaN,689943,NaN,0,3,NaN,NaN,Colectivo Specialist,TABLET Nº 2 MAGDALENA,MENÚ ACCIONA,101,2024-02-08 14:36:17,50094964,12164,999,NaN,95089774,5286,0.0,1,32017,NaN,0.0,10676,109,NaN,20240208
21499,NaN,1,NaN,689149,NaN,0,3,NaN,NaN,Colectivo Specialist,Central,Recarga de Saldo Fichero,101,2024-02-08 14:07:44,50094931,999,999,NaN,0,5288,-0.5,32002,32002,NaN,0.0,10673,105,NaN,20240208
21500,NaN,1,NaN,689150,NaN,0,3,NaN,NaN,Colectivo Specialist,TABLET Nº 6 ACCIONA,MENÚ ACCIONA,101,2024-02-08 14:07:44,50094931,12156,999,NaN,95089791,5288,0.5,1,32017,NaN,0.5,10673,109,NaN,20240208
21501,NaN,1,NaN,690122,NaN,0,2,NaN,NaN,Colectivo Manager,Central,Recarga de Saldo Fichero,101,2024-02-08 14:45:39,50095008,999,999,NaN,0,5293,-0.5,32002,32002,NaN,0.0,8313,105,NaN,20240208


#### Filter by 

In [30]:
prodelfi = prodelfi.groupby('date')['id1'].nunique().reset_index(name='occupancy_restaurant')
prodelfi

,date,occupancy_restaurant
0,20240201,1567
1,20240205,1744
2,20240206,1776
3,20240207,1673
4,20240208,1636


## Final dataset (preview)

In [31]:
df = pd.merge(workplaces, num_reservations, left_on='date', right_on='date', how='outer')
df = pd.merge(df, occupancy_total, left_on='date', right_on='date', how='outer')
df = pd.merge(df, occupancy_workplace, left_on='date', right_on='date', how='outer')
df = pd.merge(df, exp_occupancy_restaurant, left_on='date', right_on='date', how='outer')
df = pd.merge(df, prodelfi, left_on='date', right_on='date', how='outer')
df

,date,capacity,workplace_reservations,occupancy_total,occupancy_workplace,exp_occup_restaurant,occupancy_restaurant
0,20240201,3462,2174.0,2589,1986,2146,1567.0
1,20240202,3462,2148.0,2317,1861,1971,NaN
2,20240203,3462,NaN,61,25,37,NaN
3,20240204,3462,NaN,44,12,22,NaN
4,20240205,3462,2141.0,2657,2071,2202,1744.0
5,20240206,3462,2166.0,2695,2090,2245,1776.0
6,20240207,3462,2184.0,2658,2015,2157,1673.0
7,20240208,3462,2171.0,2613,1977,2123,1636.0


# 2. Final dataset

### Internal data

EmployeeEvents contiene solo los accesos de empleados (no sirve para occupancy total)

Para occupancy total -> OccupancyMesena.csv

#### Capacity

Number of reservable spaces in Mesena

In [32]:
capacityMesena = pd.read_csv(f"{path}CapacitySpaces.csv")
# Rename occupancy count column
capacityMesena = capacityMesena.rename(columns={'sum': 'capacity'})
capacityMesena[549:]

,fec_fecha,capacity
549,2024-02-01,3462
550,2024-02-02,3462
551,2024-02-03,3462
552,2024-02-04,3462
553,2024-02-05,3462
554,2024-02-06,3462
555,2024-02-07,3462
556,2024-02-08,3464
557,2024-02-09,3571
558,2024-02-10,3571


In [33]:
# first date for capacity value
capacityMesena[0:1]

,fec_fecha,capacity
0,2022-08-01,843


#### Reservations

Number of employees who reserve a workplace in Mesena

In [34]:
reservationsMesena = pd.read_csv(f"{path}ReservationsMesena.csv")
# Rename occupancy count column
reservationsMesena = reservationsMesena.rename(columns={'numemployees': 'reserv_workplace'})
reservationsMesena[406:]

,fec_fecha,reserv_workplace
406,2024-02-01,2174
407,2024-02-02,2148
408,2024-02-05,2141
409,2024-02-06,2166
410,2024-02-07,2184
411,2024-02-08,2171
412,2024-02-09,2153
413,2024-02-12,2175
414,2024-02-13,2196
415,2024-02-14,2207


In [35]:
# first date for number of reservations
reservationsMesena[0:1]

,fec_fecha,reserv_workplace
0,2022-08-01,287


#### Total occupancy

"Raw" occupancy: count of all users who enter Mesena

In [36]:
occupancyMesena = pd.read_csv(f"{path}OccupancyMesena.csv")
# Rename occupancy count column
occupancyMesena = occupancyMesena.rename(columns={'count': 'occupancy'})
occupancyMesena[539:]

,fec_evento_local,occupancy
539,2024-02-01,2589
540,2024-02-02,2317
541,2024-02-03,61
542,2024-02-04,44
543,2024-02-05,2657
544,2024-02-06,2695
545,2024-02-07,2658
546,2024-02-08,2613
547,2024-02-09,2242
548,2024-02-10,72


In [37]:
# first date for occupancy value
occupancyMesena[0:1]

,fec_evento_local,occupancy
0,2022-08-10,315


#### Workplace Occupancy 

Occupancy of employees with workplace permission

In [38]:
permissions_all = pd.read_csv(f"{path}UserPermissions.csv")
# permissions_all

In [39]:
employees_events = pd.read_csv(f"{path}EmployeesEvents.csv")
# employees_events

In [40]:
occupancy_df = pd.merge(employees_events, permissions_all, 
                          left_on=['fec_evento', 'empleado'], 
                          right_on=['fec_fecha', 'id_empleado'], 
                          how='left')
# occupancy_df

In [41]:
all_workplaces = occupancy_df[occupancy_df['flg_reserva'] == True]
occupancy_workplaces = all_workplaces.groupby('fec_evento')['empleado'].count().reset_index(name='occup_workplaces')

occupancy_workplaces[530:538]

,fec_evento,occup_workplaces
530,2024-02-01,1986
531,2024-02-02,1861
532,2024-02-03,25
533,2024-02-04,12
534,2024-02-05,2071
535,2024-02-06,2090
536,2024-02-07,2015
537,2024-02-08,1977


In [42]:
# first date for workplace occupancy
occupancy_workplaces[0:1]

,fec_evento,occup_workplaces
0,2022-08-10,248


#### Expected Restaurant Occupancy 

Occupancy of employees with restaurant permission

In [43]:
all_restaurant = occupancy_df[occupancy_df['flg_comedor'] == True]
exp_occupancy_restaurant = all_restaurant.groupby('fec_fecha')['flg_comedor'].count().reset_index(name='exp_occup_restaurant')
exp_occupancy_restaurant[537:]

,fec_fecha,exp_occup_restaurant
537,2024-02-01,2146
538,2024-02-02,1971
539,2024-02-03,37
540,2024-02-04,22
541,2024-02-05,2202
542,2024-02-06,2245
543,2024-02-07,2157
544,2024-02-08,2123
545,2024-02-09,1869
546,2024-02-10,45


In [44]:
exp_occupancy_restaurant[0:1]

,fec_fecha,exp_occup_restaurant
0,2022-08-10,260


#### Restaurant Occupancy

Prodelfi

In [45]:
prodelfiMesena = pd.read_csv(f"{path}ProdelfiMesena.csv")
prodelfiMesena = prodelfiMesena.rename(columns={'count': 'occup_restaurant'})
prodelfiMesena[205:210]

,fec_movimiento,occup_restaurant
205,2024-02-01,1567
206,2024-02-05,1744
207,2024-02-06,1776
208,2024-02-07,1673
209,2024-02-08,1636


In [46]:
prodelfiMesena[0:1]

,fec_movimiento,occup_restaurant
0,2023-01-18,40


#### Internal dataset

In [47]:
final_df = pd.merge(capacityMesena, reservationsMesena, left_on='fec_fecha', right_on='fec_fecha', how='outer')
final_df = pd.merge(final_df, occupancyMesena, left_on='fec_fecha', right_on='fec_evento_local', how='outer')
final_df.drop(columns=['fec_evento_local'], inplace=True)
final_df = pd.merge(final_df, occupancy_workplaces, left_on='fec_fecha', right_on='fec_evento', how='outer')
final_df.drop(columns=['fec_evento'], inplace=True)
final_df = pd.merge(final_df, exp_occupancy_restaurant, left_on='fec_fecha', right_on='fec_fecha', how='outer')
final_df = pd.merge(final_df, prodelfiMesena, left_on='fec_fecha', right_on='fec_movimiento', how='outer')
final_df.drop(columns=['fec_movimiento'], inplace=True)
final_df.dropna(subset=['fec_fecha'], inplace=True)
final_df.dropna(subset=['occupancy'], inplace=True)
final_df

,fec_fecha,capacity,reserv_workplace,occupancy,occup_workplaces,exp_occup_restaurant,occup_restaurant
9,2022-08-10,843.0,251.0,315.0,248.0,260.0,NaN
10,2022-08-11,843.0,247.0,294.0,227.0,238.0,NaN
11,2022-08-12,843.0,227.0,237.0,178.0,189.0,NaN
12,2022-08-13,843.0,NaN,4.0,1.0,1.0,NaN
13,2022-08-14,843.0,NaN,2.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...
599,2024-03-22,3830.0,2162.0,2080.0,1627.0,1741.0,NaN
600,2024-03-23,3830.0,NaN,70.0,24.0,37.0,NaN
601,2024-03-24,3830.0,NaN,57.0,15.0,24.0,NaN
602,2024-03-25,3830.0,1805.0,1548.0,1124.0,1228.0,954.0


### Merge with open-source data

In [48]:
opendata_path = "OPEN_DATA_FILES_PATH"

In [49]:
# Read open data df
opendata_df = pd.read_csv(f"{opendata_path}OpenData_CalendarAll_df.csv")
# opendata_df

In [50]:
data = pd.merge(final_df, opendata_df, left_on='fec_fecha', right_on='date', how='left')
data.drop(columns=['date','id_date'], inplace=True)
data = data.rename(columns={'fec_fecha': 'date'})
data

,date,capacity,reserv_workplace,occupancy,occup_workplaces,exp_occup_restaurant,occup_restaurant,day_month,month,year,day_week,day_type,weather_code,temperature_2m_max,temperature_2m_min,precipitation_sum
0,2022-08-10,843.0,251.0,315.0,248.0,260.0,NaN,10,8,2022,3,0,80.0,36.502502,21.402500,3.7
1,2022-08-11,843.0,247.0,294.0,227.0,238.0,NaN,11,8,2022,4,0,51.0,34.902500,21.502500,0.5
2,2022-08-12,843.0,227.0,237.0,178.0,189.0,NaN,12,8,2022,5,1,2.0,36.102500,22.202500,0.0
3,2022-08-13,843.0,NaN,4.0,1.0,1.0,NaN,13,8,2022,6,2,51.0,30.402500,19.602499,0.5
4,2022-08-14,843.0,NaN,2.0,1.0,1.0,NaN,14,8,2022,7,2,3.0,31.702500,16.302500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,2024-03-22,3830.0,2162.0,2080.0,1627.0,1741.0,NaN,22,3,2024,5,1,3.0,25.575998,10.976000,0.0
590,2024-03-23,3830.0,NaN,70.0,24.0,37.0,NaN,23,3,2024,6,2,3.0,23.876000,10.626000,0.0
591,2024-03-24,3830.0,NaN,57.0,15.0,24.0,NaN,24,3,2024,7,2,3.0,22.325998,11.526000,0.0
592,2024-03-25,3830.0,1805.0,1548.0,1124.0,1228.0,954.0,25,3,2024,1,1,63.0,13.376000,9.076000,7.0


### Missing values

In [51]:
data.isnull().sum()

date                      0
capacity                  0
reserv_workplace        157
occupancy                 0
occup_workplaces          9
exp_occup_restaurant      2
occup_restaurant        358
day_month                 0
month                     0
year                      0
day_week                  0
day_type                  0
weather_code              0
temperature_2m_max        0
temperature_2m_min        0
precipitation_sum         0
dtype: int64

##### Data imputing for nulls in weekends and holidays

In [52]:
data.loc[data['day_type'] == 2, 'reserv_workplace'] = data.loc[data['day_type'] == 2, 'reserv_workplace'].fillna(0)
print('Nulls in resev_workplace after imputing:', data['reserv_workplace'].isnull().sum())

Nulls in resev_workplace after imputing: 0


In [53]:
data.loc[data['day_type'] == 2, 'occup_workplaces'] = data.loc[data['day_type'] == 2, 'occup_workplaces'].fillna(0)
print('Nulls in occup_workplaces after imputing:', data['occup_workplaces'].isnull().sum())

Nulls in occup_workplaces after imputing: 0


##### Data imputing for restaurant occupancy

Substituting nulls by 0 in restaurant occupancy for weekends, holidays and Fridays (restaurant not opened)

In [54]:
data.loc[data['day_type'] == 2, 'occup_restaurant'] = data.loc[data['day_type'] == 2, 'occup_restaurant'].fillna(0)
data.loc[data['day_week'] == 5, 'occup_restaurant'] = data.loc[data['day_week'] == 5, 'occup_restaurant'].fillna(0)
print('Nulls in occup_restaurant after imputing:', data['occup_restaurant'].isnull().sum())

Nulls in occup_restaurant after imputing: 85


### Storing data

In [55]:
path = "PATH_TO_STORE"

In [56]:
data.to_csv(f"{path}final_df_all.csv", index=False)